# 1. Import module requirement

## 1.1 Import standard Python modules

In [1]:
from collections import OrderedDict, defaultdict
import pandas as pd

## 1.2 Import openalea packages


In [2]:
from openalea.deploy.shared_data import shared_data
from openalea.mtg import *
from openalea.mtg.algo import orders
from openalea.mtg import stat
import openalea.plantgl.all as pgl

NOT using graph editor observer No module named grapheditor


In [3]:
from openalea.core import path
from openalea.mtg import *
from openalea.deploy.shared_data import shared_data
import glob

## 1.3 Import strawberry modules

In [4]:
import openalea.strawberry

from openalea.strawberry import Rules_production
from openalea.strawberry import Visualization, visualisation2d
from openalea.strawberry import analysis, variables

# 2. Import data and read MTG in files

## 2.1 Import data

La cellule ci dessous sert à changer les données. Il est donc necessaire de lui dire quelles sont les données que l'on veut importer

In [5]:
%gui qt4

files = shared_data(openalea.strawberry).glob('*.mtg')
# print files

genotypes = dict((f.namebase, f) for f in files)
#print(genotypes.keys())

DIPLOIDS = ['Sicile','Nils']
OCTOPLOIDS = ['Gariguette','Darselect','Clery','Capriss','Cir107','Ciflorette','friendlyfruit']

diploids = dict((k,f) for k,f in genotypes.items() if k in DIPLOIDS)
# print(diploids)

octoploids = dict((k,f) for k,f in genotypes.items() if k in OCTOPLOIDS)
print(octoploids)

{u'friendlyfruit': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\friendlyfruit.mtg'), u'Ciflorette': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Ciflorette.mtg'), u'Darselect': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Darselect.mtg'), u'Gariguette': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Gariguette.mtg'), u'Capriss': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Capriss.mtg'), u'Clery': Path(u'c:\\users\\mlabadie\\appdata\\local\\continu

## 2.2 Load one octoploid and one diploid  genotype 

A set of strawberry plant architecture has been measured and is stored in an MTG.

- We select on octoploid and/or one diploid genotype for architectural analysis.


Pour cela replacer mettre le nom de la variété voulue oct_genotype= 'nomvariete' attention les guillemets sont importante

In [6]:
#OCTOPLOIDS = ['Gariguette_NEW','Darselect_New','Clery_New','Capriss_New''Cir107_NEW','Ciflorette_New']
#DIPLOIDS = ['Sicile_New','Nils_New.mtg']

di_genotype = 'Sicile'
oct_genotype = 'friendlyfruit'

# gdi = MTG(diploids[di_genotype])
# print('Diploid  : ', str(gdi))

goct = MTG(octoploids[oct_genotype])
print('Octoploid  : ', str(goct))

replace all the date format by -
('Octoploid  : ', 'MTG : nb_vertices=4599, nb_scales=4')


* Check number of plant present in MTG

In [7]:
# Diploid
#nb_plants = gdi.nb_vertices(scale=1)
#print(nb_plants)

# Octoploid
nb_plants = goct.nb_vertices(scale=1)
print(nb_plants)

40


## 2.3 Load whole octoploids or diploids in one big MTG

A set of strawberry plant architecture has been measured and is stored in an MTG.

- We select whole octoploids genotype for architectural analysis.

In [30]:
big_mtg = MTG()
for fn in OCTOPLOIDS:
    big_mtg = algo.union(big_mtg,MTG(octoploids[fn]))

g_octo = big_mtg
    
genotypes = set(g_octo.property("Genotype").values())
print genotypes

replace all the date format by -
replace all the date format by -
replace all the date format by -
replace all the date format by -
replace all the date format by -
replace all the date format by -
set(['Ciflorette', 'Darselect', 'Gariguette', 'Capriss', 'Clery', 'Cir107'])


- We select whole diploids genotype for architectural analysis.

In [ ]:
big_mtg = MTG()
# Add missing properties
for fn in DIPLOIDS:
    big_mtg = algo.union(big_mtg,MTG(diploids[fn]))

big_mtg.properties()['FLWRNUMBER']={}
big_mtg.properties()['FLWRNUMBER_OPEN']={}

g_diplo = big_mtg
    
genotypes = set(g_diplo.property("Genotype").values())
print genotypes

# 3. Strawberry Visualisation

## 3.1. Select MTG data for visualisation

- one genotype (goct or gdi)
- all genotypes (g_diplo or g_octo)

In [8]:
g=goct

## 3.2.  Launch 3D-visualisation

## TODO: Generale faire un argument qui me permet d'entre le nombre de plant par modalité et le nombre de modalité,
le bute étant de définir sur la grille le nombre de colone (modalité et ligne plant par modalité

In [9]:
g.properties()['order'] = orders(g)
Rules_production.color_code(g)
scene = Visualization.visualise_plants(g)

pgl.Viewer.display(scene)

c:\users\mlabadie\appdata\local\continuum\miniconda2\envs\strawberry\lib\site-packages\openalea.strawberry-1.0.0-py2.7.egg\openalea\strawberry\Rules_production.py:64: Warning: Invalid Color value 8 in setColor (maximum is 6)
  t.setColor(6+order)
c:\users\mlabadie\appdata\local\continuum\miniconda2\envs\strawberry\lib\site-packages\openalea.strawberry-1.0.0-py2.7.egg\openalea\strawberry\Rules_production.py:64: Warning: Invalid Color value 7 in setColor (maximum is 6)
  t.setColor(6+order)
c:\users\mlabadie\appdata\local\continuum\miniconda2\envs\strawberry\lib\site-packages\openalea.strawberry-1.0.0-py2.7.egg\openalea\strawberry\Rules_production.py:112: Warning: Invalid Color value 9 in setColor (maximum is 6)
  t.setColor(8+order)
c:\users\mlabadie\appdata\local\continuum\miniconda2\envs\strawberry\lib\site-packages\openalea.strawberry-1.0.0-py2.7.egg\openalea\strawberry\Rules_production.py:64: Warning: Invalid Color value 10 in setColor (maximum is 6)
  t.setColor(6+order)
c:\users\m

## 3.3.  Launch 2D-Visualisation

* 2D Visualization of one or a set of plants

Pour visualisation 2D:
* visualisation2d.plot2d(g, g.vertices(scale=1)[:x],dist= [y]*z) (cf ci dessous)

* arguments de la foncion:
  - [:x]  x= il sagit du nombre de plante que l'on veut représenter
  - dist= [y] x= espacement entre deux plante
  - z = nombre de plante visualisé

## TODO: Probleme avec la fonction on ne peut pas visualiser l'ensemble des plant

In [25]:
visualisation2d.plot2d(g, g.vertices(scale=1)[:40],dist= [10]*40) 

# 4. Strawberry analysis

## 4.1 Function to run the notebook on diploid or octoploid (only for one genotype)

In [10]:
# Run the notebook on Diploid or Octoploid

# Dictionnary to convert propertie names in order to run old version script
convert = dict(Stade='Stade', 
               Fleurs_ouverte='FLWRNUMBER_OPEN',
               Fleurs_avorte='FLWRNUMBER_ABORTED',
               Fleurs_total='FLWRNUMBER',
               date='Sample_date',
               Plante='Plant_ID',
              )

## 4.2 Extraction of data in dataframe

### 4.2.1 Extraction on plant scale

* Extraction of the most central individuals, variables (plant_variables) used;
    * Plant
    * Genotype
    * date
    * nb_total_leaves
    * nb_total_flowers
    * stolons
    * order_max
    * nb_ramification

##### TODO: Colone modalité qui permettrais de différencié les lots. 
Faire également dans la fonction calcul de l'individu central intégré une modulation par rapport à la modalité

In [12]:
# extraction of data at plant scale
df = variables.extract_at_plant_scale(g)
df.date = pd.to_datetime(df.date, dayfirst=True)



#Data of the central plants
df_individuals = analysis.median_individuals(df)
df_individuals = df_individuals.sort_values(by=['Genotype','date'])

df_individuals
df

{'_line': 1345, 'index': 1289, 'scale': 3, 'vid': 1289, 'parent': 1287, 'color': (255, 0, 255), 'complex': 1288, 'label': 'f', 'edge_type': '+', 'order': 5} {'_line': 1302, 'index': 1246, 'vid': 1246, 'scale': 1, 'Genotype': 'Gariguette', 'parent': None, 'Plant_ID': 4, 'label': 'P', 'Experiment_name': 'DOUVILLE', 'complex': 0, 'Sample_date': '26-04-2018', 'order': 0}
{'_line': 2885, 'index': 2827, 'scale': 3, 'vid': 2827, 'parent': 2825, 'color': (255, 0, 255), 'complex': 2826, 'label': 'f', 'edge_type': '+', 'order': 5} {'_line': 2806, 'index': 2748, 'vid': 2748, 'scale': 1, 'Genotype': 'Gariguette', 'parent': None, 'Plant_ID': 4, 'label': 'P', 'Experiment_name': 'ANJOU1', 'complex': 0, 'Sample_date': '26-04-2018', 'order': 0}


,Genotype,date,plant,nb_total_leaves,nb_total_flowers,nb_stolons,order_max,nb_ramifications,vid
0,Gariguette,2018-04-26,8,25,13,0,4,2,1
1,Gariguette,2018-04-26,5,20,12,0,4,1,124
2,Gariguette,2018-04-26,7,21,12,0,4,1,210
3,Gariguette,2018-04-26,1,20,24,0,4,1,314
4,Gariguette,2018-04-26,3,17,10,0,3,1,396
5,Gariguette,2018-04-26,4,25,13,0,4,2,465
6,Gariguette,2018-04-26,6,15,15,0,3,1,572
7,Gariguette,2018-04-26,2,28,13,0,4,1,650
8,Gariguette,2018-04-26,6,27,14,0,5,1,733
9,Gariguette,2018-04-26,7,32,5,0,3,2,817


* 2D visualisation of the most central individuals

In [13]:
# selection of vid of median individuals
pids = list(df_individuals.vid)
n = len(pids)

# Plot 2D- visualisation of the most central plant
visualisation2d.plot2d(g, pids, dist=[5]*n)


### 4.2.2 Data extraction on module scale

**Variables used (module_variables):**
- nb_visible_leaves # Nombre de feuille developpe
- nb_foliar_primordia #Nombre de primordia foliaire
- nb_total_leaves #Nombre total de feuille
- nb_open_flowers #Nombre de fleurs ouverte
- nb_aborted_flowers #Nombre de fleurs avorté
- nb_total_flowers #Nombre total de Fleurs
- nb_vegetative_buds
- nb_initiated_buds
- nb_floral_buds
- nb_stolons
- type_of_crown # Type de crowns (Primary Crown:1, Branch crown:2 extension crown:3)
- Crown_status

In [ ]:
# extraction of data at plant scale
df = variables.extract_at_module_scale(g)
df

## 4.3 Data sequence extractions

### 4.3.1 Module scale

* **Index:**
    - real Order (Index) for recouvrant extraction
    - Apparent Order for non recouvrant extraction

* **Variables:**
    - nb_visible_leaves : count F label in components
    - nb_foliar_primordia: count f si le module commence par F
    - nb_total_leaves: sum(F+f) du module
    - Stolon Numbers: count s label in components and attach at the order -1 
    - nb_open_flowers: g.property('Fleurs_ouverte') at scale 3
    - nb_aborted_flowers: g.property('Fleurs_avorte') at scale 3
    - nb_total_flowers : g.property('Fleurs_total') at scale 3
    - No_vegetative_bud: count bt if g.property(Stade) == none or 17 or 18 or 19
    - No_iniated_bud: count bt if g.property(Stade) == A
    - No_Floral_bud: count ht
    - No_Inflorescence: count HT
 
 
* **Variable qualitative:**
    - type of crown
        - label == T (1) : principal crown
        - parent(component_roots()[0]) : if successor() == F : branch_crown (3)
        - elif in (HT, ht, bt) extension crown (2)
    - inflorescence_type:
        - vegetative (1): label==bt g.property(Stade)== none or 17 or 18 or 19
        - initiated (2): label== bt if g.property(Stade) == A
        - floral (3): label== ht or HT


* **General Variables:**
    - Genotype: g.property('Genotype') at scale 1
    - Date: g.property('date') at scale 1
    - Plante Number: g.property('plante') at scale 1

In [ ]:
# Run the notebook on Diploid or Octoploid
IS_DIPLOID = False

if IS_DIPLOID:
    g = g_diplo
else:
    g = g_octo



print(g.property_names())
convert = dict(Stade='Stade', 
               Fleurs_ouverte='FLWRNUMBER_OPEN',
               Fleurs_avorte='FLWRNUMBER_ABORTED',
               Fleurs_total='FLWRNUMBER',
               date='Sample_date',
               Plante='Plant_ID',
              )

# if not IS_DIPLOID:
#     g.properties()['Sample_date'] = g.property('Architecture_date')

def dates_diploid():
    l = '11-05-2016', '17-05-2016', '01-06-2016', '20-06-2016'
    return dict(zip(l, range(1, len(l)+1)))

def dates_octoploid():
    d = {'10-12-2014':1,'07-01-2015':2,'15-02-2015':3,'02-03-2015':4,'03-04-2015':5,'27-05-2015':6,'04-12-2014':1,'13-02-2015':3,
         '30-03-2015':5,'08-01-2015':2,'11-02-2015':3,'04-03-2015':4,'02-04-2015':5,'20-05-2015':6,'09-01-2015':2,'06-03-2015':4,
         '12-02-2015':3,'19-05-2015':6}
    return d

def dates():
    if IS_DIPLOID:
        return dates_diploid()
    else: 
        return dates_octoploid()


def property(g, name):
    """ We can change the name of the MTG properties without changing the code"""
    return g.property(convert.get(name, name))
    

In [ ]:
# apparent_module: all module which contains a visible leaf F
def visible_modules(g=g):
    return [v for v in g.vertices_iter(scale=2) if g.label(g.component_roots_iter(v).next()) == 'F']

In [ ]:
# add property in MTG, apparant module (True) or not
modules = visible_modules()
_visible = {}
for m in modules:
    _visible[m] = True
g.properties()['visible'] = _visible

In [ ]:
"""Index

TODO: Replace dict by functions with specific traversal.

"""
#order scale 2: order at module scale
orders = algo.orders(g, scale=2)

"""Variable quantitative"""
#function which count all F
def nb_visible_leaves(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid)=='F')

#function which count all f
def nb_foliar_primordia(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid)=='f')

#function which count all f+F
def nb_total_leaves(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid) in ('f', 'F'))

""" nb_stolon"""
#function count stolon
def nb_stolons(v, g=g):
    def nb_stolon(vid, g=g):
        return sum(1 for cid in g.components(vid) if g.label(cid)=='s') 
    return sum(nb_stolon(ch) for ch in g.children(v))

#function return number of open flower
def nb_open_flowers(vid, g=g):
    flowers = property(g, 'Fleurs_ouverte')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

# function return number of aborted flower
def nb_aborted_flowers(vid, g=g):
    flowers = property(g, 'Fleurs_aborted')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

# function return number of total flower
def nb_total_flowers(vid, g=g):
    flowers = property(g, 'Fleurs_total')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))


"""Return the No vegetative bud
    
Algorithm:
if labet is bt then stage is 17,18,19 or None
count number of bt and attach at the parent order
    """
# function return number of vegetative buds
def nb_vegetative_buds(vid, g=g):
    stages= property(g, 'Stade')

    def nb_vegetative(v):
        cid = g.component_roots(v)[0]
        
        return sum(1 for cid in g.components(v) if g.label(cid)=='bt' and stages.get(cid) in (None,'17','18','19'))
    
    return sum(nb_vegetative(ch) for ch in g.children(vid))

 
""" Return the No initiated bud"""

def nb_initiated_buds(vid, g=g):
    
    stages= property(g, 'Stade')
    
    def nb_init(v):
        return sum(1 for cid in g.components(v) if (g.label(cid)=='bt') and (stages.get(cid)=='A'))

    return sum(nb_init(ch) for ch in g.children(vid))


""" Return the No Floral bud"""
def nb_floral_buds (vid, g=g):
    visibles = property(g, 'visible')
    def nb_floral(v):
        return sum(1 for cid in g.components(v) if g.label(cid)=="ht" )
    return sum(nb_floral(ch) for ch in g.children(vid) if ch not in visibles)


""" Qualitative variables"""
def type_of_crown(vid, g=g):
    """ Returns the type of crown.
    
    Definition of type of crown (1, 2, 3):
     - principal crown (1): label == T 
     - branch_crown (3)
         parent(component_roots()[0]) : if successor() == F  
     - extension_crown (2): contains(HT, ht, bt) 
     - error (4)
     
    """
    if g.label(vid) == 'T':
        return 1
    else:
        cid = g.component_roots_iter(vid).next()
        pid = g.parent(cid)
        sid = g.Successor(pid)
        #print sid
        if g.label(sid) in ('F', 'f'):
            return 3
        elif g.label(sid) in ('bt', 'ht', 'HT'):
            return 2
        else:
            # ERROR !!!
            print g[cid], g[g.complex_at_scale(cid, scale=1)]
            return 4
            
def Crown_status(vid, g=g):
    """ Returns the type of inflorescence
    
    :Algorithms:
    if label is bt then
        - if stage is 17, 18, 19 or None, => vegetative (1)
        - if stage is A => initiated (2)
        - if stage is other => non defined (pourri, avorté, coupé) (-1)
     - Terminal vegetative bud (1): label==bt g.property(Stade)== none or 17 or 18 or 19
     - Terminal initiated bud (2): label== bt if g.property(Stade) == A
     - Terminal Floral bud (3): label==ht
     - Inflorescence Terminal (4): label== HT
     - runner (5): label = s
     
    """
    stages = property(g,'Stade')
    # select s, ht, HT et bt
    for cid in g.components(vid):
        if g.label(cid) in ('s', 'ht', 'HT', 'bt'):
            label = g.label(cid)
            if label == 'ht':
                return 3
            elif label == 'HT':
                return 4
            elif label == 'bt':
                stage = stages.get(cid)
                if stage == 'A':
                    return 2
                elif stage in (None, '17', '18', '19'):
                    return 1
            elif label == 's':
                return 5
            return -1

def nb_inflorescence (Vid, g):
    return sum(1 for cid in g.components(Vid) if g.label(cid)=='HT')
    
def genotype(vid, g=g):
    d = {'Capriss':4, 'Ciflorette':2, 'Cir107':6, 'Clery':3, 'Darselect':5, 'Gariguette':1,
         'Nils': 1, }
    
    cpx = g.complex_at_scale(vid, scale=1)
    _genotype = property(g, 'Genotype')[cpx]
    return d[_genotype]

def plant(vid, g=g):
    
    cpx = g.complex_at_scale(vid, scale=1)
    return property(g, 'Plante')[cpx]

def date(vid, g=g):
    d = dates()
    cpx = g.complex_at_scale(vid, scale=1)
    _date = property(g, 'date')[cpx]
    return(d[_date])

In [ ]:
# g.property("Genotype")

In [ ]:
# Extract sequences

#seqs = stat.extract_extremities(g, scale=2)
#seqs = stat.extract_axes(g, scale=2)
def is_extremities(g, v):
    visibles = property(g, 'visible')
    ext = [c for c in g.children(v) if c in visibles and not is_axis_root(g, c)]
    return True if not ext else False

def extract_extremities(g, modules):
    """ Extract sequences that connect from leaves modules to the root ones of each plant. """
    leaves = [m for m in modules if is_extremities(g, m)]
    seqs = [list(reversed([vid for vid in algo.ancestors(g,lid)])) for lid in leaves]
    return seqs

def is_axis_root(g, vid):
    cid = g.component_roots_iter(vid).next()
    pid = g.parent(cid)
    sid = g.Successor(pid)
    if g.label(sid) not in ('bt', 'ht', 'HT'):
        return True
    else:
        return False
    

def apparent_axis(g, vid):
    visibles = property(g, 'visible')
    v = vid
    while v is not None:
        yield v
        vtx = v; v = None
        for vid in g.children(vtx):
            if (vid in visibles) and (not is_axis_root(g, vid)):
                v = vid
            
def extract_apparent_axes(g, modules):
    # Extract all the vertices with edge_type == '+'
    vids = [m for m in modules if is_axis_root(g, m)]
    
    seqs = [ list(apparent_axis(g, vid)) for vid in vids]
    return seqs


In [ ]:
seqs1 = extract_apparent_axes(g, modules)
seqs2= extract_extremities(g,modules)
seqs = seqs1
len (seqs)

In [ ]:
# Index: order
VertexIdentifiers = [['-'.join(str(orders[j]) for j in seq[0:i+1]) for i, vid in enumerate(seq)] for seq in seqs]

# Variables
variable = OrderedDict()
variable['nb_visible_leaves'] = nb_visible_leaves # Nombre de feuille developpe
variable['nb_foliar_primordia'] = nb_foliar_primordia #Nombre de primordia foliaire
variable['nb_total_leaves'] = nb_total_leaves #Nombre total de feuille
variable['nb_open_flowers'] = nb_open_flowers #Nombre de fleurs ouverte
variable['nb_aborted_flowers'] = nb_aborted_flowers #Nombre de fleurs avorté
variable['nb_total_flowers'] = nb_total_flowers #Nombre total de Fleurs
variable['vegetative_bud'] = nb_vegetative_buds
variable['Initiated_bud']= nb_initiated_buds
variable['floral_bud']= nb_floral_buds
variable['stolons']= nb_stolons
variable['type_of_crown'] = type_of_crown # Type de crowns (Primary Crown:1, Branch crown:2 extension crown:3)
variable['Crown_status'] = Crown_status # Status du coeurs ("vegeatatif:1, Iniated: 2, Floral: 3)


variable['genotype'] = genotype # TODO
variable['date'] = date # TODO
variable['plant'] = plant

In [ ]:
variable_names = variable.keys()
print variable_names

final_seqs = [[[f(vid) for f in variable.values()] for vid in seq] for seq in seqs]

In [ ]:
final_seqs

In [ ]:
sequence_txt = analysis.write_sequences(final_seqs, variable_names, VertexIdentifiers)

In [ ]:
print sequence_txt

In [ ]:
f = open('c:/Users/mlabadie/Documents/GitHub/strawberry/Rscript/strawberry_archi_module_nonoverlaping_path.seq', 'w') #seq1
f.write(sequence_txt)
f.close()